<a href="https://colab.research.google.com/github/iesous-kurios/powerBi/blob/master/PowerBI_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title
# import libraries which provide math functions to our data imports
import statistics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xlwt 
from xlwt import Workbook
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib
from matplotlib import style

sns.__version__

'0.10.0'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#@title
# import weekly data from yearly FPS folder
df = pd.read_excel('/content/drive/My Drive/FPData/YTD Data/FP_YTD_Mar_25.xlsx')
#df = pd.read_excel ('~/environments/fp_env/family_promise_data/data/ClientTrackExport (5).xlsx')

In [0]:
# get list of currently enrolled guests (guests that don't have an exit date)
currently_enrolled = df[df['3.11 Exit Date'].isnull()]

In [64]:
currently_enrolled['3.3 Birthdate'].value_counts()

2018-01-01    2
2008-10-12    1
2008-04-03    1
2007-06-22    1
1981-07-04    1
             ..
1980-01-01    1
1988-03-04    1
2018-04-09    1
1993-01-01    1
2018-12-21    1
Name: 3.3 Birthdate, Length: 130, dtype: int64

In [0]:
bdays_date = (date.today()-timedelta(days=15)).isoformat()
current_bdays = currently_enrolled[currently_enrolled['Date of Last ES Stay (Beta)'] >= bdays_date ]

In [0]:
bday_df = current_bdays[['3.1 FirstName', '3.1 LastName', '3.3 Birthdate','5.8 Personal ID']].copy()

In [0]:
import datetime
currentMonth = date.today().month
currentMonth
two_months = date.today().month + 2
next_month
last_month = date.today().month - 1

In [0]:
bday_df['month'] = pd.DatetimeIndex(bday_df['3.3 Birthdate']).month
bday_df['day'] = pd.DatetimeIndex(bday_df['3.3 Birthdate']).day

In [0]:
upcomming_bdays = bday_df[(bday_df['month'] > last_month) & (bday_df['month'] < two_months) ]

In [0]:
upcomming_bdays.to_excel('/content/drive/My Drive/FPData.close_bdays.xlsx')

In [131]:
upcomming_bdays

,3.1 FirstName,3.1 LastName,3.3 Birthdate,5.8 Personal ID,month,day
509,Alan,Kroiss,1996-03-01,89515,3,1
600,Michael,Kelly,1991-03-19,98295,3,19
644,Kharina,Madison,2018-04-09,83939,4,9
662,Amber,Shay,1988-03-04,98767,3,4
663,Devin,Shay,2018-03-25,98768,3,25


In [0]:
# create new list of those currently enrolled who haven't stayed with us for
# two weeks or more

from datetime import date, timedelta

current_date = date.today().isoformat()   
days_before = (date.today()-timedelta(days=30)).isoformat()

need_to_exit = currently_enrolled[currently_enrolled['Date of Last ES Stay (Beta)'] < days_before ]

In [0]:
exit_df = need_to_exit[['3.1 FirstName', '3.1 LastName', '5.8 Personal ID', '5.9 Household ID', '3.15 Relationship to HoH', 'CaseMembers', '3.10 Enroll Date', 'Date of Last ES Stay (Beta)']].copy()

In [0]:
#exit_df.head()

In [0]:
exit_df.to_excel('/content/drive/My Drive/FPData/YTD Data/YTDexit_df.xlsx')

In [10]:
households = df['5.9 Household ID'].unique()
len(households)

225

In [11]:
# print total number of families served (unduplicated)
unique_households = df['5.9 Household ID'].nunique()
unique_households

225

In [0]:
#@title
# replace zeros in data with "Nan" to calculate more accurate average bednights 
df['Count of Bed Nights (Housing Check-ins)'] = df['Count of Bed Nights (Housing Check-ins)'].replace(0, np.NaN)

In [13]:
avg_bn = df['Count of Bed Nights (Housing Check-ins)'].mean()
avg_bn = round(avg_bn, 2)
avg_bn

36.42

In [0]:
#@title

exit_reasons = ['Rental by client with RRH or equivalent subsidy', 
                'Rental by client, no ongoing housing subsidy', 
                'Staying or living with family, permanent tenure', 
                'Rental by client, other ongoing housing subsidy',
                'Permanent housing (other than RRH) for formerly homeless persons', 
                'Staying or living with friends, permanent tenure', 
                'Owned by client, with ongoing housing subsidy', 
                'Staying or living with friends, permanent tenure', 
                'Rental by client, VASH housing Subsidy'
               ]

temp_reasons = ['Staying or living with family, temporary tenure (e.g., room, apartment or house)', 
               'Staying or living with friends, temporary tenure (e.g., room, apartment or house)', 
                
               ]

In [0]:
# drop rows with no exit destination (current guests at time of report)
df = df.dropna(subset=['3.12 Exit Destination'])

In [0]:
 # create target column (multiple types of exits to perm)
df['perm_leaver'] = df['3.12 Exit Destination'].isin(exit_reasons)

In [0]:
#df.head()

In [0]:
perm_leaver = df[df['perm_leaver'] == True]

In [0]:
total_perm = perm_leaver.perm_leaver.value_counts().sum()

In [20]:
total_perm

189

In [0]:
non_perm = df[df['perm_leaver'] == False]

In [0]:
total_non_perm = non_perm.perm_leaver.value_counts().sum()

In [23]:
total_non_perm

400

In [0]:
perm_percent = total_perm / total_non_perm

In [25]:
perm_percent = round(perm_percent, 2)*100
perm_percent

47.0

In [26]:
df['3.12 Exit Destination'].value_counts()

Client refused                                                                                                                   165
Emergency shelter, including hotel or motel paid for with emergency shelter voucher, or RHY-funded Host Home shelter             123
Rental by client, no ongoing housing subsidy                                                                                      71
Rental by client with RRH or equivalent subsidy                                                                                   69
Transitional Housing for homeless persons (including homeless youth)                                                              36
Rental by client, other ongoing housing subsidy                                                                                   28
No exit interview completed                                                                                                       23
Staying or living with family, permanent tenure                      

In [0]:
unknowns = ['Client refused', 'No exit interview completed']

In [0]:
unknown_df = df[df['3.12 Exit Destination'].isin(unknowns)]

In [0]:
unknown_exits = unknown_df['3.12 Exit Destination'].value_counts().sum()

In [30]:
unknown_exits

188

In [31]:
unknown_percent = unknown_exits / total_non_perm
unknown_percent = round(unknown_percent, 2) *100
unknown_percent

47.0

In [32]:
exits = df['3.12 Exit Destination'].value_counts().sum()
exits

589

In [0]:
exits = df['3.12 Exit Destination'].value_counts()

In [34]:
# take sum of ES and Trans exits and divide by total exits, then multiply by 100 to get percentage
esExits = exits['Emergency shelter, including hotel or motel paid for with emergency shelter voucher, or RHY-funded Host Home shelter'] 
#transExits = ((exits['Transitional Housing for homeless persons (including homeless youth)'] / exits.sum()) * 100)
# round the answer to nearest hundreth place
esExits = (round(esExits, 2))
#transExits = (round(transExits, 2))
esExits
#print('Exits to Transitional',transExits,'%')

123

In [35]:
tempExits = df['3.12 Exit Destination'].isin(temp_reasons).sum()

tempExits

35

In [36]:
transitional = df['3.12 Exit Destination'] == 'Transitional Housing for homeless persons (including homeless youth)'
transitional = transitional.sum()
transitional

36

In [37]:
exits.sum()

589

In [0]:
total_guests = total_non_perm + total_perm

In [39]:
trans_percent = (transitional / exits.sum())*100
trans_percent = round(trans_percent, 2)
trans_percent

6.11

In [0]:
#zips_master

In [41]:
# export zips into an excel sheet for Power BI visual
zips_master = pd.read_excel('/content/drive/My Drive/FPData/uszips.xlsx')



# Import pandas library 
import pandas as pd 
  
# initialize list of lists 
zips_bi = df['V5 Zip'].value_counts().to_frame('Total').reset_index(level=0, inplace=False)
zips_bi = zips_bi.rename(columns={"index": "V5 Zip"})
zips_bi = zips_bi.merge(zips_master,on='V5 Zip')
zips_bi['V5 City'].value_counts()

Spokane           10
Spokane Valley     5
Great Falls        2
Mountain Home      2
Williston          1
Jensen Beach       1
Deer Park          1
Kennewick          1
Dubuque            1
Pullman            1
Salt Lake City     1
Hoodsport          1
Glendale           1
Bakersfield        1
Ligonier           1
Puyallup           1
Tacoma             1
Anderson           1
Alexandria         1
Airway Heights     1
Colville           1
Moscow             1
Wellpinit          1
Lolo               1
Tillamook          1
Ritzville          1
Omak               1
Celina             1
Clarkston          1
Hemet              1
Ogden              1
Vancouver          1
Marrero            1
Milwaukee          1
Browning           1
Chewelah           1
Newport            1
Nampa              1
South Bend         1
Santa Rosa         1
Kettle Falls       1
Everett            1
Liberty Lake       1
Greenacres         1
Sprague            1
Olympia            1
Medical Lake       1
Cheney       

In [42]:
type(zips_bi)

pandas.core.frame.DataFrame

In [0]:
zips_bi.to_excel('/content/drive/My Drive/FPData/YTD Data/weeklyYTDzips.xlsx', header=True, index=False)

In [0]:
other_reasons = ['Place not meant for habitation (e.g., a vehicle, an abandoned building, bus/train/subway station/airport or anywhere outside)',
                 'Hotel or Motel paid for without Emergency Shelter Voucher', 'Other', 'Substance Abuse Treatment or Detox Center',
                 'Foster Care Home or Foster Care Group Home', 'Psychiatric Hospital or Other Psychiatric Facility']

In [0]:
other = df['3.12 Exit Destination'].isin(other_reasons)

In [46]:
other.sum()
other = other.sum()
other

18

In [47]:
df['4.04.7 Private - Employer']

0       No
1       No
2       No
3       No
4       No
      ... 
661    NaN
677     No
678     No
679     No
680     No
Name: 4.04.7 Private - Employer, Length: 589, dtype: object

In [48]:
df['3.12 Exit Destination'].value_counts()

Client refused                                                                                                                   165
Emergency shelter, including hotel or motel paid for with emergency shelter voucher, or RHY-funded Host Home shelter             123
Rental by client, no ongoing housing subsidy                                                                                      71
Rental by client with RRH or equivalent subsidy                                                                                   69
Transitional Housing for homeless persons (including homeless youth)                                                              36
Rental by client, other ongoing housing subsidy                                                                                   28
No exit interview completed                                                                                                       23
Staying or living with family, permanent tenure                      

In [49]:
print(total_guests, 'total guests served YTD')
print(total_non_perm, 'total non perm leavers')
print(total_perm, 'total perm leavers')
print(esExits, 'total ES leavers')
print(tempExits, 'total temp leavers')
print(transitional, 'total trans leavers')
print(other, 'total leavers to "other"')
print(unknown_exits, 'total unknown leavers')
print('sum of all groups to make sure it matches total served:',total_perm+esExits+tempExits+transitional+other+unknown_exits)

589 total guests served YTD
400 total non perm leavers
189 total perm leavers
123 total ES leavers
35 total temp leavers
36 total trans leavers
18 total leavers to "other"
188 total unknown leavers
sum of all groups to make sure it matches total served: 589


In [0]:
assert((total_perm+esExits+tempExits+transitional+other+unknown_exits) == (total_non_perm+total_perm))

In [0]:
perm_bi = '{} %'.format(perm_percent)
es_bi = '{} %'.format(esExits)
temp_bi = '{} %'.format(tempExits)
trans_bi = '{} %'.format(trans_percent)
unknown_bi = '{} %'.format(unknown_percent)

In [0]:
power_bi = pd.DataFrame({'Permanent Leavers': total_perm, 'Emergency Shelter Leavers': esExits, 
                         'Temporary Leavers': tempExits, 'Transitional Leavers': transitional, 
                         'Unknown Leavers': unknown_exits, 'Other Leavers': other, 'Total': total_guests }, index=[0])


In [0]:
# Import pandas library 
import pandas as pd 
  
# initialize list of lists 
data = [['Permanent Leavers', total_perm], ['Emergency Shelter Leavers', esExits], 
        ['Temporary Leavers', tempExits], ['Transitional Leavers', transitional],
        ['Unknown Leavers', unknown_exits], ['Other Leavers', other]] 
  


# Create the pandas DataFrame 
power_bi = pd.DataFrame(data, columns = ['Exit Destination', 'Total']) 
  
# print dataframe. 

In [54]:
power_bi

,Exit Destination,Total
0,Permanent Leavers,189
1,Emergency Shelter Leavers,123
2,Temporary Leavers,35
3,Transitional Leavers,36
4,Unknown Leavers,188
5,Other Leavers,18


In [55]:
power_bi.Total.sum()

589

In [0]:
power_bi.to_excel('/content/drive/My Drive/FPData/YTD Data/YTDweeklyDataNew.xlsx')

In [58]:
for col in df:
    print(col)

4.04.8 Health Insurance obtained through COBRA
4.04.7 Private - Employer
4.04.9 Private
4.04.9 Private - Individual
4.2.3a Earned Income
4.2.4b Unemployment Insurance
4.2.5c Supplemental Security Income
4.2.6d Social Security Disability  Income
4.2.7e VA Disability Compensation
4.2.8f VA Disability Pension
4.2.9g Private Disability Income
4.2.10h Workers Compensation
4.2.11i TANF
4.2.12j General Assistance
4.2.13k Retirement (Social Security)
4.2.14l Pension from a Former Job
4.2.15m Child Support
4.2.16n Alimony
4.2.17o Other Income
Chronic Homeless Status_vHMISDatAssessment
Chronic Homeless Status_EvaluatevHMIS&HMISDA
Email
HomePhone
WorkPhone
2.1 Organization Name
2.2 Project Name
2.4 ProjectType
2.5 Utilization Tracking Method (Invalid)
2.6 Federal Grant Programs
Enrollment Created By
3.1 FirstName
3.1 LastName
5.8 Personal ID
5.9 Household ID
3.15 Relationship to HoH
3.16 Client Location
CaseMembers
3.10 Enroll Date
3.11 Exit Date
3.12 Exit Destination
3.2 SocSecNo
3.2 Social Secu

In [59]:
df['3.12 Exit Destination'].value_counts()

Client refused                                                                                                                   165
Emergency shelter, including hotel or motel paid for with emergency shelter voucher, or RHY-funded Host Home shelter             123
Rental by client, no ongoing housing subsidy                                                                                      71
Rental by client with RRH or equivalent subsidy                                                                                   69
Transitional Housing for homeless persons (including homeless youth)                                                              36
Rental by client, other ongoing housing subsidy                                                                                   28
No exit interview completed                                                                                                       23
Staying or living with family, permanent tenure                      